# Setting up the Notebook

KBMOD uses Python's logging library for output during the run. If you are interested in running with debug output (verbose mode), you can set the level of Python's logger to `WARNING` (for warning messages only), `INFO` (for moderate output), or `DEBUG` (for comprehensive output). By default logging is set to warning level.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

# Load the data.

We load the data using the `ImageCollection` class to load all the FITS files in the given directory.

`KBMODV0_5Config` provides some parameters about how to read the files. The important ones for this investiation are the ones involved with masking:

* bit_flag_map is a dictionary from masking reason to the bit set in the mask layer, such as: `{ "BAD": 2**0, "EDGE": 2**4, "NO_DATA": 2**8, "UNMASKEDNAN": 2**15 }`
* mask_flags is a list of which reasons to mask such as `["BAD", "UNMASKEDNAN"]`
    
The combination of those two flags will mask all pixels with the 1st and/or 15th bits set in the mask layer.

Below we are not masking anything because I don't know which bits correspond to which flags for this survery.

In [ ]:
filepath = "/astro/users/jkubica/dwarf_star_data"

from kbmod.image_collection import ImageCollection
from kbmod.standardizers.fits_standardizers.kbmodv05 import KBMODV0_5, KBMODV0_5Config


load_config = KBMODV0_5Config(mask_flags=[])

# Loading from the files takes a while (multiple minutes).
ic = ImageCollection.fromDir(filepath, force=KBMODV0_5, config=load_config)
wu = ic.toWorkUnit()
wu.print_stats()

INFO:kbmod.image_collection:Creating ImageCollection from 29 standardizers.
INFO:kbmod.image_collection:Building WorkUnit from ImageCollection


WorkUnit:
  Num Constituent Images (29):
  Reprojected: False
Image Stack Statistics:
  Image Count: 29
  Image Size: 6295 x 6261 = 39412995
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|  idx |     Time   |  Flux Min  |  Flux Max  |  Flux Mean |  Var Min |  Var Max | Var Mean | Masked |
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|    0 |  55677.544 |    -129.07 |   50170.41 |       2.64 |    -0.02 |     0.03 |     0.01 |  86.42 |
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|    1 |  56404.531 |    -154.93 |   57970.41 |       2.30 |     0.00 |     0.02 |     0.01 |  51.92 |
+------+------------+------------+------------+------------+----------+----------+----------+--------+
|    2 |  54985.429 |    -285.37 |  109745.98 |       2.88 |     0.00 |     0.00 |     0.00 |  24.04 |
+------+------------+------------+-

If you want to modify the images directly, you can use the `im_stack` attribute in the WorkUnit.  For example, we could use built-in `ImageStack` functions to mask out science pixels that too bright or negative variance pixels.

In [ ]:
wu.im_stack.mask_by_science_bounds(max_val=10000.0)
wu.im_stack.mask_by_variance_bounds(min_val=1e-10)

wu.print_stats()

We could even plot the first image. The image stack contains lists of images `sci` for science and `var` for variance. So we would access the first science image as `wu.im_stack.sci[0]`

In [ ]:
from kbmod.analysis.plotting import plot_image

plot_image(wu.im_stack.sci[0], norm=True)

# Set up the KBMOD Search

The standard approach to running KBMOD is to perform a grid search over all starting pixels and a grid of velocities. Let’s do a grid search with:
* 21 different velocity steps from -2 pixels per day and 2 pixels per day in the x dimension
* 21 different velocity steps from -2 pixels per day and 2 pixels per day in the y dimension

KBMOD needs a series of configuration parameters to specify all the information about the search. For the full list of parameters see:

https://epyc.astro.washington.edu/~kbmod/user_manual/search_params.html

In this notebook we explicitly provide the configuration parameters as a dictionary so users can see what is being specified. However most users will want to use the `SearchConfiguration` class. A `SearchConfiguration` object uses reasonable defaults when created:

In [ ]:
from kbmod.configuration import SearchConfiguration

input_parameters = {
    # Use search parameters (including a force ecliptic angle of 0.0)
    # to match what we know is in the demo data.
    "generator_config": {
        "name": "VelocityGridSearch",
        "vx_steps": 21,
        "min_vx": -2.0,
        "max_vx": 2.0,
        "vy_steps": 21,
        "min_vy": -2.0,
        "max_vy": 2.0,
    },
    # Output parameters
    "result_filename": "./results.ecsv",
    # Basic filtering (always applied)
    "num_obs": 15,  # <-- Filter anything with fewer than 15 observations
    "lh_level": 10.0,  # <-- Filter anything with a likelihood < 10.0
    # SigmaG clipping parameters
    "sigmaG_lims": [25, 75],  # <-- Clipping parameters (lower and upper percentile)
    # Other parameters
    "cpu_only": True,  # <-- This will be absurdly slow.  Set to False if you have a good enough GPU.
}
config = SearchConfiguration.from_dict(input_parameters)

# Make the WorkUnit use this configuration.
wu.config = config

## More Limited Search

Before doing a full search across the images, I would suggest starting with a very limited search around the known object's position and velocity.  Something like:

In [ ]:
# Substitute known values here if you have them.
known_x0 = 50  # The object's x pixel coordinate at the first time.
known_y0 = 70  # The object's y pixel coordinate at the first time.
known_vx = 100.0  # The object's x velocity in pixels per day.
known_vy = 20.0  # The object's y velocity in pixels per day.

input_parameters = {
    "x_pixel_bounds": [known_x0 - 10, known_x0 + 10],
    "y_pixel_bounds": [known_y0 - 10, known_y0 + 10],

    # Use search parameters (including a force ecliptic angle of 0.0)
    # to match what we know is in the demo data.
    "generator_config": {
        "name": "VelocityGridSearch",
        "vx_steps": 11,
        "min_vx": known_vx - 2.0,
        "max_vx": known_vx + 2.0,
        "vy_steps": 11,
        "min_vy": known_vy - 2.0,
        "max_vy": known_vy + 2.0,
    },
    # Output parameters
    "result_filename": "./results.ecsv",
    # Basic filtering (always applied)
    "num_obs": 15,  # <-- Filter anything with fewer than 15 observations
    "lh_level": 10.0,  # <-- Filter anything with a likelihood < 10.0
    # SigmaG clipping parameters
    "sigmaG_lims": [25, 75],  # <-- Clipping parameters (lower and upper percentile)
    # Other parameters
    "cpu_only": True,  # <-- This will be absurdly slow.  Set to False if you have a good enough GPU.
}
config = SearchConfiguration.from_dict(input_parameters)

# Make the WorkUnit use this configuration.
wu.config = config

# Running KBMOD

You run KBMOD by giving it a WorkUnit that contains the input image data and the configuration.

In [ ]:
from kbmod.run_search import SearchRunner

rs = SearchRunner()
results = rs.run_search_from_work_unit(wu)

In [ ]:
print(results)